In [ ]:
!pip install sklearn_crfsuite
!pip install git+https://www.github.com/keras-team/keras-contrib.git
!pip install tensorflow==1.15.0 keras==2.2.4
!pip3 install glove-python-binary

In [ ]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

from nltk.corpus import stopwords
import pandas as pd
import pickle
import re
import datetime
import os
import numpy as np

In [ ]:
import nltk
nltk.download("stopwords")

In [ ]:
## ADDITIONAL FUNCTIONS
if not os.path.exists('models'):
    os.makedirs('models')
    
pStopWordsList = stopwords.words('english') + stopwords.words('russian')

def convert_amount(pAmount, rubles = False):
    try:
        if rubles:
            cAmount = pytils.numeral.rubles(abs(float(pAmount)), zero_for_kopeck=True)    
        else:
            cAmount = pytils.numeral.in_words(abs(int(pAmount)))    
    except Exception as e:
        cAmount = pAmount  
    return cAmount

def convert_date(pDate, format_mode, formatDate = None):
    formatDates = ['%Y-%m-%d',
                   '%d-%m-%Y',
                   '%d/%m/%Y',
                   '%d.%m.%Y',
                   '%y-%m-%d',
                   '%d-%m-%y',
                   '%d/%m/%y',
                   '%d.%m.%y',
                   '%Y-%m-%d %H:%M:%S',
                   '%d-%m-%Y %H:%M:%S',
                   '%d/%m/%Y %H:%M:%S',
                   '%d.%m.%Y %H:%M:%S',
                   '%y-%m-%d %H:%M:%S',
                   '%d-%m-%y %H:%M:%S',
                   '%d/%m/%y %H:%M:%S',
                   '%d.%m.%y %H:%M:%S']
    if formatDate!=None:
        cDate = datetime.datetime.strptime(pDate, formatDate).date()
    else:
        for fd in formatDates:
            try:
                cDate = datetime.datetime.strptime(pDate, fd).date()
                break
            except Exception as e:
                cDate=None
    if cDate == None:
        print('convert_date: Can not define date format for "%s"' % pDate)
        return ''
    if format_mode == 1:
        dmonth = {1: '01', 2: '02', 3: '03', 4: '04', 5: '05', 6: '06', 7: '07', 8: '08', 9: '09', 10: '10', 11: '11', 12: '12'}
        dformat = '{}.{}.{}'
        res = dformat.format(cDate.day, dmonth[cDate.month], cDate.year - 2000)
    if format_mode == 2:
        dmonth = {1: 'января', 2: 'февраля', 3: 'марта', 4: 'апреля', 5: 'мая', 6: 'июня', 7: 'июля', 8: 'августа', 9: 'сентября', 10: 'октября', 11: 'ноября', 12: 'декабря'}   
        dformat = '{} {} {}'
        res = dformat.format(cDate.day, dmonth[cDate.month], cDate.year)
    if format_mode == 3:
        dmonth = {1: 'января', 2: 'февраля', 3: 'марта', 4: 'апреля', 5: 'мая', 6: 'июня', 7: 'июля', 8: 'августа', 9: 'сентября', 10: 'октября', 11: 'ноября', 12: 'декабря'}   
        dformat = '"{}" {} {}'
        res = dformat.format(str(cDate.day).zfill(2), dmonth[cDate.month], cDate.year)            
    if format_mode == 4:
        dmonth = {1: '01', 2: '02', 3: '03', 4: '04', 5: '05', 6: '06', 7: '07', 8: '08', 9: '09', 10: '10', 11: '11', 12: '12'}
        dformat = '{}.{}.{}'
        res = dformat.format(cDate.day, dmonth[cDate.month], cDate.year)            
    return res

def checkdate(text, pattern):
    res = ''
    if pattern==None: return ''
    text = text.lower()
    pattern = pattern.lower()
    res = pattern
    if res in text: return res
    res = convert_date(pattern, format_mode=1).lower()
    if res in text: return res
    res = convert_date(pattern, format_mode=4).lower()
    if res in text: return res   
    res = res.replace('.','/')
    if res in text: return res   
    res = res.replace('/','-')
    if res in text: return res       
    res = convert_date(pattern, format_mode=2).lower()
    if res in text: return res
    res = convert_date(pattern, format_mode=3).lower()
    if res in text: return res
    res = res[1:]
    if res in text: return res
    res = res.replace('""', '')
    if res in text: return res
    
    if " ".join([w for w in convert_date(pattern, format_mode=1).lower()]) in text:
        res = " ".join([w for w in convert_date(pattern, format_mode=1)])
    if " ".join([w for w in convert_date(pattern, format_mode=2).lower()]) in text:
        res = " ".join([w for w in convert_date(pattern, format_mode=2)])
    
    return ''
    
def checkamount(text, pattern, space =3):
    digits = ["0","1","2","3","4","5","6","7","8","9"]
    if pattern in (None,''): return ''
    res = ''
    
    if pattern in (None,'','nan'): return ''
    
    if pattern == 'nan':
        return res
    if pattern in text:
        res = pattern
    res=pattern
    if res in text: return res
    
    pattern = pattern.lower().replace(' ','').replace(',', '.')#.replace('-', '')
    res = pattern
    if res in text: return res
    
    res = pattern.replace('.', ',')
    if res in text: return res
    
    res = pattern.replace(',', '.')
    if res in text: return res

    res = pattern.replace(',', '-')
    if res in text: return res

    res = pattern.replace('.', '-')
    if res in text: return res
    
    try:
        res = '{:_}'.format(float(pattern)).replace('_', ' ')
        if res in text: return res
        res = res.replace('.',',')
        if res in text: return res
        
        res = res.replace('.','-')
        if res in text: return res
    except Exception as e:
        tmp = None
    
    res = convert_amount(pattern).lower()
    if convert_amount(pattern, rubles=True).lower() in text:
    
        res = convert_amount(pattern, rubles=True).lower()        
    if res in text: return res
        
    tmp = pattern.split('.')
    res = convert_amount(tmp[0]).lower()
    tmp[0] = '{:,}'.format(int(tmp[0])).replace(',', ' ')
    
    res = ','.join(tmp)
    if res in text: return res
    
    res = '.'.join(tmp)
    if res in text: return res
    
    reg = ''
    for l in pattern:
        if l in digits:
            reg = '%s%s.{0,%s}' % (reg, l,space)
    m = re.search(reg, text)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-space]

    for w in pattern.split():
        reg = '%s%s.{0,%s}' % (reg, w,space)
    m = re.search(reg, text)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-space]
    
    return ''

def checkstring(text, target, spaces = 2):
    if target in (None,'','nan'): return ''
    text = text.lower()
    target = target.lower()
    '''
    reg = ''
    for l in target:
        reg = '%s%s.{0,%s}' % (re.escape(reg), l,spaces)
    
    m = re.search(reg, text)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
    '''
    target = target.replace('ооо','').replace('ао бывш.зао', '').\
             replace('зао','').replace('оао','').replace('ао','').\
             replace('ип','').replace(',','').strip()
    if target in text:
        return target
    if target in ['бн', 'б/н']:
        return ''
    if target.replace('0','o') in text.replace('0','o'):
        return target.replace('0','o')
    if target.replace('з','3') in text.replace('з','3'):
        return target.replace('з','3')    
    if target.replace('э','3') in text.replace('э','3'):
        return target.replace('э','3')    
    if target.replace('э','9') in text.replace('э','9'):
        return target.replace('э','9')    
    if target.replace('б','6') in text.replace('б','6'):
        return target.replace('б','6')    
    for ch in [' ','_', '\\','`','*','{','}','[',']','(',')','>','#','+','-','.','!','$','\'', ' ']:
        if ch in target:
            target = target.replace(ch,"")
            if target in text:
                return target
    return ''

def process_text_layer(TextLayer, pStopWordsList = []):
    txt =  " ".join([w for w in TextLayer.lower().split() \
                     if (not w in pStopWordsList)])

    txt = txt. \
        replace('«', ''). \
        replace('»', ''). \
        replace('(', ''). \
        replace(')', ''). \
        replace('\[)', ''). \
        replace('\]', ''). \
        replace('^', ''). \
        replace('\\', '')

    return txt

def tagging_sentence_by_targets(ptag, sentence_tagged, df_targets, pmodel_name):
    ret = sentence_tagged
    for r in df_targets[df_targets['tag']==ptag]['string_value']:
        tagging_sentence(ret, r, pmodel_name)
    
    return ret

def word2features2(sent, i, exclude = 'O', bias=3):
    word = sent[i][0]
    if sent[i][2] == exclude:
        return None

    features = {
        'bias': bias,
        'word.lower()': word.lower(),
        'word[:3]': word[:3],
        'word[:2]': word[:2],
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isdigit()': word.isdigit(),
        'BOS': False,
        'EOS': False
    }
    for w in range(1, bias+1):
        if i > w:
            word1 = sent[i-w][0]
            features.update({
                '-%s:word.lower()' % (w): word1.lower()
            })
            
    if i < 1:
        features['BOS'] = True

    for w in range(1, bias+1):
        if i < len(sent)-w:
            word1 = sent[i+w][0]
            features.update({
                '+%s:word.lower()' % (w): word1.lower()
            })
            
    if i >= len(sent)-1:
        features['EOS'] = True

    return features

def sent2features(sent, exclude = 'O'):
    result = list()
    for i in range(len(sent)):
        ret = word2features2(sent, i, exclude = exclude, bias=10)
        if ret !=None:
            result.append(ret)
    return result

def sent2labels(sent, exclude = 'O'):
    result = list()
    for token, postag, label in sent:
        if label !=exclude:
            result.append(label)        
    return result

def tagging_sentence(sentence, target, tag):
    sentence_text=  ' '.join([w['Word'] for w in sentence])
    r_tag = ''

    try:
        res_find  = re.finditer(target.lower(), sentence_text.lower())
    except:
        return sentence

    for m in res_find:
        r_tag = 'B-%s' % tag
        for i in range(len(sentence_text[:m.start()].split()), len(sentence_text[:m.end()].split())):
            """
            if r_tag==('B-%s' % tag):
                if i>0:
                    sentence[i-1]['Tag']= 'B1-%s' % tag
                if i>1:
                    sentence[i-2]['Tag']= 'B2-%s' % tag                      
                if i>2:
                    sentence[i-3]['Tag']= 'B3-%s' % tag
                if i>3:
                    sentence[i-4]['Tag']= 'B4-%s' % tag  
            """
            sentence[i]['Tag']= r_tag
            r_tag = 'I-%s' % tag   
            """
            for j in range(1,5):
                if i+j < len(sentence):
                    sentence[i+j]['Tag']='E%s-%s' % (str(j), tag)
            """
    return sentence

def prepare_sentence(num, sentence):
    words = sentence.replace('"', '').split()
    tmp = []
    for w in words:
        tmp.append({'Sentence #': 'Sentence: %s' % num, 'Word' : w, 'POS' : 'NN', 'Tag': 'O'})
    return tmp

In [ ]:
!gdown --id 1hYI4ovDNu2jUUBOMAEKya6BPbYEBd5An

In [ ]:
data = pd.read_csv('NER PAD.csv')
for column in data.columns:
    data[column] = data[column].astype(str)
data['DOCCPTYINN'] = data['DOCCPTYINN'].apply(lambda r: r[:-2] if type(r) == str else r)

In [ ]:
PADData = data.copy()
PADData['string_value'] = PADData.apply(lambda r: process_text_layer(r['string_value'], pStopWordsList), axis = 1)

In [ ]:
PADData.head()

In [ ]:
models = ['DOCNUM',
            'DOCDATE',
            'DOCAMOUNT',
            'DOCCPTY',
            'DOCAGRNUM',
            'DOCAGRDATE',
            'DOCCUSTOMER',
            'DOCCPTYINN']

datasets = {}
for model_name in models:
    ds = PADData[['tag','string_value', model_name]].rename(columns={'string_value': 'x', model_name: 'target'})
    if 'DATE' in model_name:
        ds['target'] = ds.apply(lambda r: checkdate(r['x'], r['target']),axis=1).copy()
    if model_name in ['DOCNUM', 'DOCAGRNUM', 'DOCCUSTOMER', 'DOCCPTY']:
        ds['target'] = ds.apply(lambda r: checkstring(r["x"], r['target']),axis=1).copy()      
    if 'AMOUNT' in model_name:
        ds['target'] = ds.apply(lambda r: checkamount(r['x'], r['target']),axis=1).copy()
        
    ds = ds[ds['target']!=''].drop_duplicates().copy()
    ds["x_tagged"] = ds.apply(lambda r: prepare_sentence(r.name, r["x"]), axis=1)
    ds["x_tagged"] = ds.apply(lambda r: tagging_sentence(r['x_tagged'], r['target'], model_name), axis=1)
    
    datasets[model_name] = ds.copy()

In [ ]:
def choose(word, x, y):
    x = x.strip()
    y = y.strip()
    word = word.strip()
    nx = len(x)
    ny = len(y)
    n = len(word)

    find_x = -1
    find_y = -1
    if x.find(word) != -1:
        find_x = max(len(word), find_x)
    if word.find(x) != -1:
        find_x = max(len(x), find_x)
    if y.find(word) != -1:
        find_y = max(len(word), find_y)
    if word.find(y) != -1:
        find_y = max(len(y), find_y)

    if find_x != -1 or find_y != -1:
        if find_x >= find_y:
            return 0
        else:
            return 1
    
    index = 0
    while index < nx and index < ny and index < n:
        if x[index] != word[index] and y[index] == word[index]:
            return 1
        elif x[index] == word[index] and y[index] != word[index]:
             return 0
        index += 1

    if index < ny - 1 and index < n - 1 and y[index + 1] == word[index + 1]:
        return 1
    elif index < nx - 1 and index < n - 1 and x[index + 1] == word[index + 1]:
        return 0
    
    return 0 #default

def merge_tags(row):
    x = row.tags_x
    y = row.tags_y
    if type(y) != list:
        return
    if type(x) != list:
        x = y
        return
    assert len(x) == len(y)
    for i in range(len(x)):
        if x[i] == 'O' and y[i] != 'O':
            x[i] = y[i]
        if x[i] != 'O' and y[i] != 'O' and x[i] != y[i]: #labels conflict
            print(f'Labels conflict with word {row.words_x[i]} --- {x[i]} : {row.target_x}; {y[i]} : {row.target_y}')
            if pd.isnull(row.target_x):
                x[i] = y[i]
                continue
            if pd.isnull(row.target_y):
                continue
            choosed = choose(row.words_x[i], row.target_x, row.target_y)
            if choosed == 0:
                print(f'Selected : {row.target_x}')
            else:
                print(f'Selected : {row.target_y}')
                x[i] = y[i]

In [ ]:
d_merged = datasets["DOCNUM"].copy()
d_merged["tags"] = d_merged.x_tagged.apply(lambda sent : [elem['Tag'] for elem in sent])
d_merged["words"] = d_merged.x_tagged.apply(lambda sent : [elem['Word'] for elem in sent])
d_merged.head(1)

In [ ]:
for model in models[1:]:
    d_new = datasets[model].copy()
    d_new["tags"] = d_new.x_tagged.apply(lambda sent : [elem['Tag'] for elem in sent])
    d_new["words"] = d_new.x_tagged.apply(lambda sent : [elem['Word'] for elem in sent])
    d_new.head(1)
    d_merged = pd.merge(d_merged, d_new.loc[:, ["tags", "target", "words", "tag"]], left_index=True, right_index=True, how="outer")
    d_merged.loc[pd.isna(d_merged.tags_x), 'tags_x'] = d_merged.loc[pd.isna(d_merged.tags_x), 'tags_y']
    d_merged.loc[pd.isna(d_merged.words_x), 'words_x'] = d_merged.loc[pd.isna(d_merged.words_x), 'words_y']
    d_merged.loc[pd.isna(d_merged.tag_x), 'tag_x'] = d_merged.loc[pd.isna(d_merged.tag_x), 'tag_y']
    d_merged.apply(merge_tags, axis=1)
    d_merged.drop(columns=['target_y', 'tags_y', 'words_y', 'tag_y'], inplace=True)
    d_merged.rename(columns={'target_x' : 'target', 'tags_x' : 'tags', 'words_x' : 'words', 'tag_x' : 'tag'}, inplace=True)

In [ ]:
df = d_merged.loc[:, ['tag', 'words', 'tags']].rename(columns={'tag' : 'id', 'tags' : 'ner_tags', 'words' : 'tokens'})

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.to_csv("NER_PAD_agg.csv", index=False)

In [ ]:
import gensim.downloader as api

pretrained_w2v = api.load("word2vec-ruscorpora-300")

In [ ]:
pretrained_w2v.save_word2vec_format('tmp')

In [ ]:
def add_suffix(x):
    SUFFIXES = ['_DET', '_NOUN', '_VERB', '_ADJ']
    res = []
    for w in x['tokens']:
        has_found = False
        for suffix in SUFFIXES:
            if w + suffix in pretrained_w2v.vocab.keys():
                res.append(w + suffix)
                has_found = True
                break
        if not has_found:
            res.append(w)
    return res

df['tokens_with_suffix'] = df.apply(add_suffix, axis=1)

In [ ]:
from gensim.models import Word2Vec


w2v = Word2Vec(sentences=df['tokens_with_suffix'], size=300, window=15, min_count=2)

In [ ]:
w2v.intersect_word2vec_format('tmp')

In [ ]:
all_tags = set()
for _, row in df.iterrows():
    for tag in row.ner_tags:
        all_tags.add(tag)
all_tags = sorted(list(all_tags))
tag_to_idx = {t: i for i, t in enumerate(all_tags)}
label_list = {i: t for i, t in enumerate(all_tags)}

In [ ]:
def find_token(x):
    SUFFIXES = ['_DET', '_NOUN', '_VERB', '_ADJ', '']
    res = []
    for w in x['tokens']:
        has_found = False
        for suffix in SUFFIXES:
            if w + suffix in w2v.wv.vocab.keys():
                res.append(w2v.wv.vocab[w + suffix].index)
                has_found = True
                break
        if not has_found:
            res.append(0)
    return res

df['encoded_ner_tags'] = df.apply(lambda x: [tag_to_idx[t] for t in x['ner_tags']], axis=1)
df['int_tokens'] = df.apply(find_token, axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

In [ ]:
from keras.optimizers import Adam
from keras.models import *
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Input
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy, crf_marginal_accuracy
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn_crfsuite import metrics
import tensorflow as tf

In [ ]:
def focal_loss(gamma=2., alpha=1.):

    gamma = float(gamma)
    alpha = float(alpha)

    def focal_loss_fixed(y_true, y_pred):
        """Focal loss for multi-classification
        FL(p_t)=-alpha(1-p_t)^{gamma}ln(p_t)
        Notice: y_pred is probability after softmax
        gradient is d(Fl)/d(p_t) not d(Fl)/d(x) as described in paper
        d(Fl)/d(p_t) * [p_t(1-p_t)] = d(Fl)/d(x)
        Focal Loss for Dense Object Detection
        https://arxiv.org/abs/1708.02002

        Arguments:
            y_true {tensor} -- ground truth labels, shape of [batch_size, num_cls]
            y_pred {tensor} -- model's output, shape of [batch_size, num_cls]

        Keyword Arguments:
            gamma {float} -- (default: {2.0})
            alpha {float} -- (default: {4.0})

        Returns:
            [tensor] -- loss.
        """
        epsilon = 1.e-9
        y_true = tf.convert_to_tensor(y_true, tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, tf.float32)

        model_out = tf.add(y_pred, epsilon)
        ce = tf.multiply(y_true, -tf.log(model_out))
        weight = tf.multiply(y_true, tf.pow(tf.subtract(1., model_out), gamma))
        fl = tf.multiply(alpha, tf.multiply(weight, ce))
        reduced_fl = tf.reduce_max(fl, axis=1)
        return tf.reduce_mean(reduced_fl)
    return focal_loss_fixed

### PRETRAINED W2V

In [ ]:
MAX_WORDS = len(w2v.wv.vocab)
EMBEDDING_LENGTH = w2v.vector_size
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 100

model = Sequential()
model.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[w2v.wv.vectors], trainable=True))
model.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model.add(crf)

model.compile(Adam(lr=0.005), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model.summary()

Using TensorFlow backend.





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 943, 300)          7440900   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 943, 200)          320800    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 943, 16)           3216      
_________________________________________________________________
crf_1 (CRF)                  (None, 943, 16)           560       
Total params: 7,765,476
Trainable params: 7,765,476
Non-trainable params: 0
_________________________________________________________________


In [ ]:
callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=2),
         ModelCheckpoint(filepath='best_model.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=412,
    callbacks=callbacks,
    validation_split=0.1,
)

model.load_weights('best_model.h5')




Train on 2880 samples, validate on 320 samples
Epoch 1/30
2880/2880 [==============================] - 33s 11ms/step - loss: 0.8468 - crf_marginal_accuracy: 0.0020 - val_loss: 0.6788 - val_crf_marginal_accuracy: 0.0033
Epoch 2/30
2880/2880 [==============================] - 30s 11ms/step - loss: 0.6148 - crf_marginal_accuracy: 0.0077 - val_loss: 0.5010 - val_crf_marginal_accuracy: 0.0494
Epoch 3/30
2880/2880 [==============================] - 31s 11ms/step - loss: 0.4503 - crf_marginal_accuracy: 0.5836 - val_loss: 0.3565 - val_crf_marginal_accuracy: 0.9038
Epoch 4/30
2880/2880 [==============================] - 30s 11ms/step - loss: 0.3271 - crf_marginal_accuracy: 0.9320 - val_loss: 0.2665 - val_crf_marginal_accuracy: 0.9568
Epoch 5/30
2880/2880 [==============================] - 31s 11ms/step - loss: 0.2413 - crf_marginal_accuracy: 0.9677 - val_loss: 0.2017 - val_crf_marginal_accuracy: 0.9709
Epoch 6/30
2880/2880 [==============================] - 31s 11ms/step - loss: 0.1803 - crf

KeyboardInterrupt: ignored

In [ ]:
preds = model.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
print(metrics.flat_classification_report(test['encoded_ner_tags'], truncated_preds, digits=4))

              precision    recall  f1-score   support

           0     0.0483    0.3467    0.0848       150
           1     0.3735    0.6760    0.4811       179
           2     0.0613    0.8150    0.1140      1243
           3     0.3775    0.8672    0.5260       595
           4     0.4811    0.9043    0.6281       282
           5     0.6634    0.8966    0.7626       609
           6     0.3323    0.8248    0.4737       645
           7     0.3511    0.6359    0.4525       651
           8     0.0000    0.0000    0.0000        88
           9     0.0000    0.0000    0.0000         7
          10     0.2978    0.3242    0.3104       620
          11     0.5360    0.8440    0.6556       564
          12     0.2305    0.9329    0.3696       462
          13     0.5579    0.9802    0.7110       605
          15     0.9919    0.8391    0.9091    137847

    accuracy                         0.8358    144547
   macro avg     0.3535    0.6591    0.4319    144547
weighted avg     0.9617   

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
MAX_WORDS = len(w2v.wv.vocab)
EMBEDDING_LENGTH = w2v.vector_size
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 100

model_fixed_embeds = Sequential()
model_fixed_embeds.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[w2v.wv.vectors], trainable=False))
model_fixed_embeds.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model_fixed_embeds.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model_fixed_embeds.add(crf)

model_fixed_embeds.compile(Adam(lr=0.005), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model_fixed_embeds.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 943, 300)          7440900   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 943, 200)          320800    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 943, 16)           3216      
_________________________________________________________________
crf_2 (CRF)                  (None, 943, 16)           560       
Total params: 7,765,476
Trainable params: 324,576
Non-trainable params: 7,440,900
_________________________________________________________________


In [ ]:
callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=2),
         ModelCheckpoint(filepath='best_model_fixed_embeds.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model_fixed_embeds.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=412,
    callbacks=callbacks,
    validation_split=0.1,
)

model_fixed_embeds.load_weights('best_model_fixed_embeds.h5')

Train on 2880 samples, validate on 320 samples
Epoch 1/30
2880/2880 [==============================] - 32s 11ms/step - loss: 0.7720 - crf_marginal_accuracy: 0.1673 - val_loss: 0.5614 - val_crf_marginal_accuracy: 0.1097
Epoch 2/30
2880/2880 [==============================] - 29s 10ms/step - loss: 0.4955 - crf_marginal_accuracy: 0.8009 - val_loss: 0.3783 - val_crf_marginal_accuracy: 0.9209
Epoch 3/30
2880/2880 [==============================] - 29s 10ms/step - loss: 0.3571 - crf_marginal_accuracy: 0.9326 - val_loss: 0.2913 - val_crf_marginal_accuracy: 0.9323
Epoch 4/30
2880/2880 [==============================] - 29s 10ms/step - loss: 0.2869 - crf_marginal_accuracy: 0.9396 - val_loss: 0.2446 - val_crf_marginal_accuracy: 0.9445
Epoch 5/30
2880/2880 [==============================] - 29s 10ms/step - loss: 0.2438 - crf_marginal_accuracy: 0.9471 - val_loss: 0.2134 - val_crf_marginal_accuracy: 0.9491
Epoch 6/30
2880/2880 [==============================] - 29s 10ms/step - loss: 0.2132 - crf_ma

In [ ]:
preds = model_fixed_embeds.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
print(metrics.flat_classification_report(test['encoded_ner_tags'], truncated_preds, digits=4))

              precision    recall  f1-score   support

           0     0.1165    0.6200    0.1962       150
           1     0.2046    0.6480    0.3110       179
           2     0.0738    0.8632    0.1359      1243
           3     0.2234    0.8202    0.3512       595
           4     0.1784    0.8794    0.2967       282
           5     0.4331    0.8768    0.5798       609
           6     0.3094    0.8202    0.4493       645
           7     0.1778    0.6390    0.2782       651
           8     0.3467    0.7841    0.4808        88
           9     0.0000    0.0000    0.0000         7
          10     0.2653    0.7629    0.3937       620
          11     0.4989    0.8404    0.6262       564
          12     0.4076    0.9113    0.5632       462
          13     0.6868    0.9570    0.7997       605
          15     0.9935    0.8286    0.9036    137847

    accuracy                         0.8283    144547
   macro avg     0.3277    0.7501    0.4244    144547
weighted avg     0.9612   

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### BILSTM + CRF

In [ ]:
all_tags = set()
word_to_ix = {}
for _, row in df.iterrows():
    for tag in row.ner_tags:
        all_tags.add(tag)
    for word in row.tokens:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
all_tags = sorted(list(all_tags))
tag_to_idx = {t: i for i, t in enumerate(all_tags)}
# tag_to_idx['EOS'] = len(tag_to_idx)
label_list = {i: t for i, t in enumerate(all_tags)}
# label_list[len(label_list)] = 'EOS'

In [ ]:
MAX_WORDS = len(word_to_ix)
EMBEDDING_LENGTH = 300
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 100

model = Sequential()
model.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH))
model.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
# model.add(Dense(50))
model.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model.add(crf)

model.compile(Adam(lr=0.005), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 943, 300)          19955400  
_________________________________________________________________
bidirectional_3 (Bidirection (None, 943, 200)          320800    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 943, 16)           3216      
_________________________________________________________________
crf_3 (CRF)                  (None, 943, 16)           560       
Total params: 20,279,976
Trainable params: 20,279,976
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=2),
         ModelCheckpoint(filepath='best_model.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=MAX_WORDS)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=412,
    callbacks=callbacks,
    validation_split=0.1,
)

model.load_weights('best_model.h5')

Train on 2880 samples, validate on 320 samples
Epoch 1/30
2880/2880 [==============================] - 34s 12ms/step - loss: 0.9006 - crf_marginal_accuracy: 0.0022 - val_loss: 0.7973 - val_crf_marginal_accuracy: 0.0020
Epoch 2/30
2880/2880 [==============================] - 31s 11ms/step - loss: 0.6824 - crf_marginal_accuracy: 0.0039 - val_loss: 0.5395 - val_crf_marginal_accuracy: 0.0049
Epoch 3/30
2880/2880 [==============================] - 30s 11ms/step - loss: 0.4482 - crf_marginal_accuracy: 0.2425 - val_loss: 0.3449 - val_crf_marginal_accuracy: 0.8683
Epoch 4/30
2880/2880 [==============================] - 30s 11ms/step - loss: 0.2728 - crf_marginal_accuracy: 0.9321 - val_loss: 0.2335 - val_crf_marginal_accuracy: 0.9712
Epoch 5/30
2880/2880 [==============================] - 30s 10ms/step - loss: 0.1769 - crf_marginal_accuracy: 0.9824 - val_loss: 0.1817 - val_crf_marginal_accuracy: 0.9869
Epoch 6/30
2880/2880 [==============================] - 30s 10ms/step - loss: 0.1240 - crf_ma

In [ ]:
preds = model.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
print(metrics.flat_classification_report(test['encoded_ner_tags'], truncated_preds, digits=4))

              precision    recall  f1-score   support

           0     0.7706    0.5600    0.6486       150
           1     0.7767    0.4469    0.5674       179
           2     0.8035    0.5527    0.6549      1243
           3     0.8976    0.8252    0.8599       595
           4     0.9633    0.8369    0.8956       282
           5     0.9476    0.9507    0.9492       609
           6     0.9314    0.8000    0.8607       645
           7     0.7651    0.5853    0.6632       651
           8     0.7273    0.7273    0.7273        88
           9     0.0000    0.0000    0.0000         7
          10     0.7462    0.4694    0.5762       620
          11     0.8858    0.8528    0.8690       564
          12     0.9364    0.9567    0.9465       462
          13     0.8990    0.9273    0.9129       605
          15     0.9876    0.9952    0.9914    137847

    accuracy                         0.9829    144547
   macro avg     0.8025    0.6991    0.7415    144547
weighted avg     0.9815   

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### NOT PRETRAINED W2V

In [ ]:
w2v = Word2Vec(sentences=df['tokens'], size=50, window=5, min_count=2)

In [ ]:
from keras.optimizers import Adam
from keras.models import *
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Input
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy, crf_marginal_accuracy

MAX_WORDS = len(w2v.wv.vocab)
EMBEDDING_LENGTH = w2v.vector_size
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 100

model = Sequential()
model.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[w2v.wv.vectors], trainable=True))
model.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model.add(crf)

model.compile(Adam(lr=0.005), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 943, 50)           3325900   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 943, 200)          120800    
_________________________________________________________________
time_distributed_4 (TimeDist (None, 943, 16)           3216      
_________________________________________________________________
crf_4 (CRF)                  (None, 943, 16)           560       
Total params: 3,450,476
Trainable params: 3,450,476
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=2),
         ModelCheckpoint(filepath='best_model.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=412,
    callbacks=callbacks,
    validation_split=0.1,
)

model.load_weights('best_model.h5')

Train on 2880 samples, validate on 320 samples
Epoch 1/30
2880/2880 [==============================] - 33s 11ms/step - loss: 0.8149 - crf_marginal_accuracy: 0.0515 - val_loss: 0.6067 - val_crf_marginal_accuracy: 0.0874
Epoch 2/30
2880/2880 [==============================] - 30s 10ms/step - loss: 0.5138 - crf_marginal_accuracy: 0.0860 - val_loss: 0.3813 - val_crf_marginal_accuracy: 0.1648
Epoch 3/30
2880/2880 [==============================] - 31s 11ms/step - loss: 0.3415 - crf_marginal_accuracy: 0.8348 - val_loss: 0.2754 - val_crf_marginal_accuracy: 0.9515
Epoch 4/30
2880/2880 [==============================] - 30s 10ms/step - loss: 0.2638 - crf_marginal_accuracy: 0.9558 - val_loss: 0.2267 - val_crf_marginal_accuracy: 0.9607
Epoch 5/30
2880/2880 [==============================] - 29s 10ms/step - loss: 0.2196 - crf_marginal_accuracy: 0.9632 - val_loss: 0.1952 - val_crf_marginal_accuracy: 0.9691
Epoch 6/30
2880/2880 [==============================] - 30s 10ms/step - loss: 0.1874 - crf_ma

In [ ]:
from sklearn_crfsuite import metrics

preds = model.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
print(metrics.flat_classification_report(test['encoded_ner_tags'], truncated_preds, digits=4))

              precision    recall  f1-score   support

           0     0.5714    0.7467    0.6474       150
           1     0.7325    0.6425    0.6845       179
           2     0.6867    0.6879    0.6873      1243
           3     0.8723    0.8958    0.8839       595
           4     0.8308    0.9574    0.8896       282
           5     0.9120    0.9015    0.9067       609
           6     0.8773    0.8310    0.8535       645
           7     0.8710    0.6221    0.7258       651
           8     0.6296    0.7727    0.6939        88
           9     0.0000    0.0000    0.0000         7
          10     0.6651    0.6758    0.6704       620
          11     0.8891    0.8954    0.8922       564
          12     0.9100    0.9632    0.9359       462
          13     0.8697    0.9488    0.9075       605
          15     0.9911    0.9914    0.9912    137847

    accuracy                         0.9827    144547
   macro avg     0.7539    0.7688    0.7580    144547
weighted avg     0.9827   

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
MAX_WORDS = len(w2v.wv.vocab)
EMBEDDING_LENGTH = w2v.vector_size
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 100

model_fixed_embeds = Sequential()
model_fixed_embeds.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[w2v.wv.vectors], trainable=False))
model_fixed_embeds.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model_fixed_embeds.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model_fixed_embeds.add(crf)

model_fixed_embeds.compile(Adam(lr=0.005), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model_fixed_embeds.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 943, 50)           3325900   
_________________________________________________________________
bidirectional_5 (Bidirection (None, 943, 200)          120800    
_________________________________________________________________
time_distributed_5 (TimeDist (None, 943, 16)           3216      
_________________________________________________________________
crf_5 (CRF)                  (None, 943, 16)           560       
Total params: 3,450,476
Trainable params: 124,576
Non-trainable params: 3,325,900
_________________________________________________________________


In [ ]:
callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=2),
         ModelCheckpoint(filepath='best_model_fixed_embeds.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model_fixed_embeds.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=412,
    callbacks=callbacks,
    validation_split=0.1,
)

model_fixed_embeds.load_weights('best_model_fixed_embeds.h5')

Train on 2880 samples, validate on 320 samples
Epoch 1/30
2880/2880 [==============================] - 34s 12ms/step - loss: 0.8792 - crf_marginal_accuracy: 0.0015 - val_loss: 0.7295 - val_crf_marginal_accuracy: 0.0031
Epoch 2/30
2880/2880 [==============================] - 30s 10ms/step - loss: 0.6578 - crf_marginal_accuracy: 0.0042 - val_loss: 0.5458 - val_crf_marginal_accuracy: 0.0048
Epoch 3/30
2880/2880 [==============================] - 30s 10ms/step - loss: 0.4801 - crf_marginal_accuracy: 0.0191 - val_loss: 0.3729 - val_crf_marginal_accuracy: 0.0917
Epoch 4/30
2880/2880 [==============================] - 30s 10ms/step - loss: 0.3362 - crf_marginal_accuracy: 0.6958 - val_loss: 0.2713 - val_crf_marginal_accuracy: 0.9497
Epoch 5/30
2880/2880 [==============================] - 30s 10ms/step - loss: 0.2604 - crf_marginal_accuracy: 0.9593 - val_loss: 0.2210 - val_crf_marginal_accuracy: 0.9618
Epoch 6/30
2880/2880 [==============================] - 30s 10ms/step - loss: 0.2212 - crf_ma

In [ ]:
preds = model_fixed_embeds.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
print(metrics.flat_classification_report(test['encoded_ner_tags'], truncated_preds, digits=4))

              precision    recall  f1-score   support

           0     0.1482    0.4467    0.2226       150
           1     0.0998    0.6927    0.1745       179
           2     0.0835    0.8656    0.1523      1243
           3     0.4424    0.8521    0.5824       595
           4     0.1125    0.9539    0.2012       282
           5     0.5737    0.8752    0.6931       609
           6     0.1954    0.7752    0.3121       645
           7     0.3093    0.6728    0.4238       651
           8     0.1429    0.0682    0.0923        88
           9     0.0000    0.0000    0.0000         7
          10     0.2176    0.4355    0.2902       620
          11     0.6108    0.8794    0.7209       564
          12     0.4294    0.9416    0.5898       462
          13     0.4837    0.9835    0.6485       605
          15     0.9939    0.8449    0.9134    137847

    accuracy                         0.8425    144547
   macro avg     0.3229    0.6858    0.4011    144547
weighted avg     0.9623   

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## window=5, min_count=1

In [ ]:
w2v = Word2Vec(sentences=df['tokens'], size=50, window=5, min_count=1)

In [ ]:
from keras.optimizers import Adam
from keras.models import *
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Input
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy, crf_marginal_accuracy

MAX_WORDS = len(w2v.wv.vocab)
EMBEDDING_LENGTH = w2v.vector_size
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 100

model = Sequential()
model.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[w2v.wv.vectors], trainable=True))
model.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model.add(crf)

model.compile(Adam(lr=0.005), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 943, 50)           3325900   
_________________________________________________________________
bidirectional_6 (Bidirection (None, 943, 200)          120800    
_________________________________________________________________
time_distributed_6 (TimeDist (None, 943, 16)           3216      
_________________________________________________________________
crf_6 (CRF)                  (None, 943, 16)           560       
Total params: 3,450,476
Trainable params: 3,450,476
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=2),
         ModelCheckpoint(filepath='best_model.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=412,
    callbacks=callbacks,
    validation_split=0.1,
)

model.load_weights('best_model.h5')

Train on 2880 samples, validate on 320 samples
Epoch 1/30
2880/2880 [==============================] - 35s 12ms/step - loss: 0.8562 - crf_marginal_accuracy: 0.0022 - val_loss: 0.6870 - val_crf_marginal_accuracy: 0.0049
Epoch 2/30
2880/2880 [==============================] - 30s 10ms/step - loss: 0.6179 - crf_marginal_accuracy: 0.0226 - val_loss: 0.4982 - val_crf_marginal_accuracy: 0.0525
Epoch 3/30
2880/2880 [==============================] - 30s 10ms/step - loss: 0.4442 - crf_marginal_accuracy: 0.7812 - val_loss: 0.3484 - val_crf_marginal_accuracy: 0.9453
Epoch 4/30
2880/2880 [==============================] - 30s 10ms/step - loss: 0.3204 - crf_marginal_accuracy: 0.9650 - val_loss: 0.2617 - val_crf_marginal_accuracy: 0.9765
Epoch 5/30
2880/2880 [==============================] - 30s 10ms/step - loss: 0.2471 - crf_marginal_accuracy: 0.9742 - val_loss: 0.2114 - val_crf_marginal_accuracy: 0.9749
Epoch 6/30
2880/2880 [==============================] - 30s 10ms/step - loss: 0.2009 - crf_ma

In [ ]:
from sklearn_crfsuite import metrics

preds = model.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
print(metrics.flat_classification_report(test['encoded_ner_tags'], truncated_preds, digits=4))

              precision    recall  f1-score   support

           0     0.5249    0.6333    0.5740       150
           1     0.6513    0.7095    0.6791       179
           2     0.6837    0.6766    0.6801      1243
           3     0.8642    0.8235    0.8434       595
           4     0.7695    0.9468    0.8490       282
           5     0.8869    0.9015    0.8941       609
           6     0.7531    0.8558    0.8012       645
           7     0.7811    0.6359    0.7011       651
           8     0.6804    0.7500    0.7135        88
           9     0.0000    0.0000    0.0000         7
          10     0.7193    0.6242    0.6684       620
          11     0.8384    0.8830    0.8601       564
          12     0.8433    0.9437    0.8907       462
          13     0.8910    0.9455    0.9174       605
          15     0.9904    0.9897    0.9901    137847

    accuracy                         0.9805    144547
   macro avg     0.7252    0.7546    0.7375    144547
weighted avg     0.9806   

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
MAX_WORDS = len(w2v.wv.vocab)
EMBEDDING_LENGTH = w2v.vector_size
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 100

model_fixed_embeds = Sequential()
model_fixed_embeds.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[w2v.wv.vectors], trainable=False))
model_fixed_embeds.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model_fixed_embeds.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model_fixed_embeds.add(crf)

model_fixed_embeds.compile(Adam(lr=0.005), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model_fixed_embeds.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 943, 50)           3325900   
_________________________________________________________________
bidirectional_7 (Bidirection (None, 943, 200)          120800    
_________________________________________________________________
time_distributed_7 (TimeDist (None, 943, 16)           3216      
_________________________________________________________________
crf_7 (CRF)                  (None, 943, 16)           560       
Total params: 3,450,476
Trainable params: 124,576
Non-trainable params: 3,325,900
_________________________________________________________________


In [ ]:
callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=2),
         ModelCheckpoint(filepath='best_model_fixed_embeds.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model_fixed_embeds.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=412,
    callbacks=callbacks,
    validation_split=0.1,
)

model_fixed_embeds.load_weights('best_model_fixed_embeds.h5')

Train on 2880 samples, validate on 320 samples
Epoch 1/30
2880/2880 [==============================] - 34s 12ms/step - loss: 0.7601 - crf_marginal_accuracy: 0.3145 - val_loss: 0.5484 - val_crf_marginal_accuracy: 0.1410
Epoch 2/30
2880/2880 [==============================] - 29s 10ms/step - loss: 0.4764 - crf_marginal_accuracy: 0.6969 - val_loss: 0.3612 - val_crf_marginal_accuracy: 0.9332
Epoch 3/30
2880/2880 [==============================] - 29s 10ms/step - loss: 0.3275 - crf_marginal_accuracy: 0.9488 - val_loss: 0.2543 - val_crf_marginal_accuracy: 0.9480
Epoch 4/30
2880/2880 [==============================] - 29s 10ms/step - loss: 0.2470 - crf_marginal_accuracy: 0.9527 - val_loss: 0.2050 - val_crf_marginal_accuracy: 0.9516
Epoch 5/30
2880/2880 [==============================] - 29s 10ms/step - loss: 0.2086 - crf_marginal_accuracy: 0.9589 - val_loss: 0.1780 - val_crf_marginal_accuracy: 0.9605
Epoch 6/30
2880/2880 [==============================] - 28s 10ms/step - loss: 0.1839 - crf_ma

In [ ]:
preds = model_fixed_embeds.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
print(metrics.flat_classification_report(test['encoded_ner_tags'], truncated_preds, digits=4))

              precision    recall  f1-score   support

           0     0.2512    0.6733    0.3659       150
           1     0.2531    0.6927    0.3707       179
           2     0.2091    0.8093    0.3323      1243
           3     0.2918    0.8891    0.4394       595
           4     0.2647    0.9397    0.4131       282
           5     0.5630    0.8883    0.6892       609
           6     0.2492    0.8341    0.3837       645
           7     0.2392    0.6575    0.3508       651
           8     0.5593    0.7500    0.6408        88
           9     0.0000    0.0000    0.0000         7
          10     0.2367    0.8435    0.3696       620
          11     0.3676    0.9326    0.5273       564
          12     0.3917    0.9545    0.5554       462
          13     0.6589    0.9868    0.7902       605
          15     0.9943    0.9040    0.9470    137847

    accuracy                         0.9015    144547
   macro avg     0.3687    0.7837    0.4784    144547
weighted avg     0.9637   

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## window=5, max_vocab_size=15000

In [ ]:
w2v = Word2Vec(sentences=df['tokens'], size=50, window=5, max_vocab_size=15000)

In [ ]:
from keras.optimizers import Adam
from keras.models import *
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Input
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy, crf_marginal_accuracy

MAX_WORDS = len(w2v.wv.vocab)
EMBEDDING_LENGTH = w2v.vector_size
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 100

model = Sequential()
model.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[w2v.wv.vectors], trainable=True))
model.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model.add(crf)

model.compile(Adam(lr=0.005), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 943, 50)           238250    
_________________________________________________________________
bidirectional_8 (Bidirection (None, 943, 200)          120800    
_________________________________________________________________
time_distributed_8 (TimeDist (None, 943, 16)           3216      
_________________________________________________________________
crf_8 (CRF)                  (None, 943, 16)           560       
Total params: 362,826
Trainable params: 362,826
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=2),
         ModelCheckpoint(filepath='best_model.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=412,
    callbacks=callbacks,
    validation_split=0.1,
)

model.load_weights('best_model.h5')

Train on 2880 samples, validate on 320 samples
Epoch 1/30
2880/2880 [==============================] - 35s 12ms/step - loss: 0.7767 - crf_marginal_accuracy: 0.0238 - val_loss: 0.5682 - val_crf_marginal_accuracy: 0.0854
Epoch 2/30
2880/2880 [==============================] - 29s 10ms/step - loss: 0.4952 - crf_marginal_accuracy: 0.3229 - val_loss: 0.3734 - val_crf_marginal_accuracy: 0.9122
Epoch 3/30
2880/2880 [==============================] - 29s 10ms/step - loss: 0.3420 - crf_marginal_accuracy: 0.9404 - val_loss: 0.2698 - val_crf_marginal_accuracy: 0.9545
Epoch 4/30
2880/2880 [==============================] - 29s 10ms/step - loss: 0.2632 - crf_marginal_accuracy: 0.9547 - val_loss: 0.2231 - val_crf_marginal_accuracy: 0.9610
Epoch 5/30
2880/2880 [==============================] - 30s 10ms/step - loss: 0.2213 - crf_marginal_accuracy: 0.9636 - val_loss: 0.1914 - val_crf_marginal_accuracy: 0.9632
Epoch 6/30
2880/2880 [==============================] - 30s 10ms/step - loss: 0.1928 - crf_ma

In [ ]:
from sklearn_crfsuite import metrics

preds = model.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
print(metrics.flat_classification_report(test['encoded_ner_tags'], truncated_preds, digits=4))

              precision    recall  f1-score   support

           0     0.4778    0.5733    0.5212       150
           1     0.4638    0.6089    0.5266       179
           2     0.5879    0.7104    0.6434      1243
           3     0.7704    0.8739    0.8189       595
           4     0.7745    0.9255    0.8433       282
           5     0.8788    0.8933    0.8860       609
           6     0.6628    0.8930    0.7609       645
           7     0.7367    0.6406    0.6853       651
           8     0.5649    0.8409    0.6758        88
           9     1.0000    0.2857    0.4444         7
          10     0.7239    0.6597    0.6903       620
          11     0.7681    0.9220    0.8380       564
          12     0.8222    0.9307    0.8731       462
          13     0.8381    0.9752    0.9015       605
          15     0.9913    0.9849    0.9881    137847

    accuracy                         0.9767    144547
   macro avg     0.7374    0.7812    0.7398    144547
weighted avg     0.9787   

In [ ]:
MAX_WORDS = len(w2v.wv.vocab)
EMBEDDING_LENGTH = w2v.vector_size
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 100

model_fixed_embeds = Sequential()
model_fixed_embeds.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[w2v.wv.vectors], trainable=False))
model_fixed_embeds.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model_fixed_embeds.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model_fixed_embeds.add(crf)

model_fixed_embeds.compile(Adam(lr=0.005), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model_fixed_embeds.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 943, 50)           238250    
_________________________________________________________________
bidirectional_9 (Bidirection (None, 943, 200)          120800    
_________________________________________________________________
time_distributed_9 (TimeDist (None, 943, 16)           3216      
_________________________________________________________________
crf_9 (CRF)                  (None, 943, 16)           560       
Total params: 362,826
Trainable params: 124,576
Non-trainable params: 238,250
_________________________________________________________________


In [ ]:
callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=2),
         ModelCheckpoint(filepath='best_model_fixed_embeds.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model_fixed_embeds.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=412,
    callbacks=callbacks,
    validation_split=0.1,
)

model_fixed_embeds.load_weights('best_model_fixed_embeds.h5')

Train on 2880 samples, validate on 320 samples
Epoch 1/30
2880/2880 [==============================] - 35s 12ms/step - loss: 0.8174 - crf_marginal_accuracy: 0.0037 - val_loss: 0.6230 - val_crf_marginal_accuracy: 0.0250
Epoch 2/30
2880/2880 [==============================] - 29s 10ms/step - loss: 0.5464 - crf_marginal_accuracy: 0.0336 - val_loss: 0.4215 - val_crf_marginal_accuracy: 0.0759
Epoch 3/30
2880/2880 [==============================] - 29s 10ms/step - loss: 0.3826 - crf_marginal_accuracy: 0.5616 - val_loss: 0.2946 - val_crf_marginal_accuracy: 0.9357
Epoch 4/30
2880/2880 [==============================] - 29s 10ms/step - loss: 0.2822 - crf_marginal_accuracy: 0.9436 - val_loss: 0.2282 - val_crf_marginal_accuracy: 0.9508
Epoch 5/30
2880/2880 [==============================] - 28s 10ms/step - loss: 0.2290 - crf_marginal_accuracy: 0.9565 - val_loss: 0.1956 - val_crf_marginal_accuracy: 0.9546
Epoch 6/30
2880/2880 [==============================] - 29s 10ms/step - loss: 0.2007 - crf_ma

In [ ]:
preds = model_fixed_embeds.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
print(metrics.flat_classification_report(test['encoded_ner_tags'], truncated_preds, digits=4))

              precision    recall  f1-score   support

           0     0.1647    0.7467    0.2699       150
           1     0.2095    0.7151    0.3241       179
           2     0.1477    0.8311    0.2509      1243
           3     0.3006    0.8706    0.4469       595
           4     0.2274    0.9362    0.3659       282
           5     0.3356    0.8818    0.4862       609
           6     0.1996    0.8775    0.3253       645
           7     0.2177    0.6636    0.3279       651
           8     0.5468    0.8636    0.6696        88
           9     0.0000    0.0000    0.0000         7
          10     0.3958    0.7290    0.5131       620
          11     0.6120    0.9007    0.7288       564
          12     0.6505    0.9264    0.7643       462
          13     0.6728    0.9719    0.7951       605
          15     0.9943    0.8895    0.9390    137847

    accuracy                         0.8873    144547
   macro avg     0.3783    0.7869    0.4805    144547
weighted avg     0.9642   

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## SOME MORE TO COME

In [ ]:
w2v = Word2Vec(sentences=df['tokens'], size=50, window=25, min_count=2)

In [ ]:
MAX_WORDS = len(w2v.wv.vocab)
EMBEDDING_LENGTH = w2v.vector_size
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 100

model = Sequential()
model.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[w2v.wv.vectors], trainable=True))
model.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model.add(crf)

model.compile(Adam(lr=0.005), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 943, 50)           1240150   
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 943, 200)          120800    
_________________________________________________________________
time_distributed_14 (TimeDis (None, 943, 16)           3216      
_________________________________________________________________
crf_14 (CRF)                 (None, 943, 16)           560       
Total params: 1,364,726
Trainable params: 1,364,726
Non-trainable params: 0
_________________________________________________________________


In [ ]:
callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=5),
         ModelCheckpoint(filepath='best_model.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=412,
    callbacks=callbacks,
    validation_split=0.1,
)

model.load_weights('best_model.h5')

Train on 2880 samples, validate on 320 samples
Epoch 1/30
2880/2880 [==============================] - 30s 10ms/step - loss: 0.0075 - crf_marginal_accuracy: 0.9995 - val_loss: 0.1830 - val_crf_marginal_accuracy: 0.9970
Epoch 2/30
2880/2880 [==============================] - 30s 10ms/step - loss: 0.0072 - crf_marginal_accuracy: 0.9996 - val_loss: 0.1899 - val_crf_marginal_accuracy: 0.9970
Epoch 3/30
2880/2880 [==============================] - 30s 10ms/step - loss: 0.0068 - crf_marginal_accuracy: 0.9996 - val_loss: 0.2008 - val_crf_marginal_accuracy: 0.9970
Epoch 4/30
2880/2880 [==============================] - 30s 10ms/step - loss: 0.0065 - crf_marginal_accuracy: 0.9996 - val_loss: 0.1971 - val_crf_marginal_accuracy: 0.9969
Epoch 5/30
2880/2880 [==============================] - 30s 10ms/step - loss: 0.0065 - crf_marginal_accuracy: 0.9996 - val_loss: 0.2081 - val_crf_marginal_accuracy: 0.9970
Epoch 6/30
2880/2880 [==============================] - 30s 10ms/step - loss: 0.0061 - crf_ma

In [ ]:
preds = model.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
print(metrics.flat_classification_report(test['encoded_ner_tags'], truncated_preds, digits=4))

              precision    recall  f1-score   support

           0     0.7913    0.6067    0.6868       150
           1     0.7448    0.6034    0.6667       179
           2     0.8340    0.6951    0.7582      1243
           3     0.9092    0.8756    0.8921       595
           4     0.9430    0.8794    0.9101       282
           5     0.9736    0.9097    0.9406       609
           6     0.8920    0.8326    0.8613       645
           7     0.7274    0.6436    0.6830       651
           8     0.6739    0.7045    0.6889        88
           9     1.0000    0.5714    0.7273         7
          10     0.7438    0.6274    0.6807       620
          11     0.8986    0.8954    0.8970       564
          12     0.9729    0.9329    0.9525       462
          13     0.9103    0.9388    0.9243       605
          15     0.9905    0.9946    0.9925    137847

    accuracy                         0.9852    144547
   macro avg     0.8670    0.7807    0.8175    144547
weighted avg     0.9845   

## GLOVE

In [ ]:
from glove import Corpus, Glove

#Creating a corpus object
corpus = Corpus() 

#Training the corpus to generate the co occurence matrix which is used in GloVe
corpus.fit(df['tokens'], window=15)

glove = Glove(no_components=300, learning_rate=0.05) 
glove.fit(corpus.matrix, epochs=30, no_threads=8, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.save('glove.model')

Performing 30 training epochs with 8 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29


In [ ]:
MAX_WORDS = len(glove.dictionary)
EMBEDDING_LENGTH = glove.no_components
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 100

model = Sequential()
model.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[glove.word_vectors], trainable=True))
model.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model.add(crf)

model.compile(Adam(lr=0.005), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 943, 300)          19955400  
_________________________________________________________________
bidirectional_4 (Bidirection (None, 943, 200)          320800    
_________________________________________________________________
time_distributed_4 (TimeDist (None, 943, 16)           3216      
_________________________________________________________________
crf_7 (CRF)                  (None, 943, 16)           560       
Total params: 20,279,976
Trainable params: 20,279,976
Non-trainable params: 0
_________________________________________________________________


In [ ]:
callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=5),
         ModelCheckpoint(filepath='best_model.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=412,
    callbacks=callbacks,
    validation_split=0.1,
)

model.load_weights('best_model.h5')

Train on 2880 samples, validate on 320 samples
Epoch 1/30
2880/2880 [==============================] - 37s 13ms/step - loss: 0.8392 - crf_marginal_accuracy: 0.0362 - val_loss: 0.6756 - val_crf_marginal_accuracy: 0.1628
Epoch 2/30
2880/2880 [==============================] - 32s 11ms/step - loss: 0.5870 - crf_marginal_accuracy: 0.3821 - val_loss: 0.4515 - val_crf_marginal_accuracy: 0.9718
Epoch 3/30
2880/2880 [==============================] - 32s 11ms/step - loss: 0.3902 - crf_marginal_accuracy: 0.9703 - val_loss: 0.2995 - val_crf_marginal_accuracy: 0.9740
Epoch 4/30
2880/2880 [==============================] - 32s 11ms/step - loss: 0.2448 - crf_marginal_accuracy: 0.9761 - val_loss: 0.1972 - val_crf_marginal_accuracy: 0.9810
Epoch 5/30
2880/2880 [==============================] - 33s 11ms/step - loss: 0.1562 - crf_marginal_accuracy: 0.9835 - val_loss: 0.1545 - val_crf_marginal_accuracy: 0.9877
Epoch 6/30
2880/2880 [==============================] - 33s 11ms/step - loss: 0.1122 - crf_ma

In [ ]:
preds = model.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
print(metrics.flat_classification_report(test['encoded_ner_tags'], truncated_preds, digits=4))

              precision    recall  f1-score   support

           0     0.7788    0.5867    0.6692       150
           1     0.8350    0.4804    0.6099       179
           2     0.8074    0.6002    0.6885      1243
           3     0.9067    0.8655    0.8856       595
           4     0.9567    0.8617    0.9067       282
           5     0.9729    0.9442    0.9583       609
           6     0.9109    0.8403    0.8742       645
           7     0.9169    0.5929    0.7201       651
           8     0.7108    0.6705    0.6901        88
           9     1.0000    0.5714    0.7273         7
          10     0.7646    0.4661    0.5792       620
          11     0.9060    0.8883    0.8970       564
          12     0.9775    0.9394    0.9581       462
          13     0.9135    0.9421    0.9276       605
          15     0.9885    0.9960    0.9922    137847

    accuracy                         0.9847    144547
   macro avg     0.8897    0.7497    0.8056    144547
weighted avg     0.9836   

In [ ]:
MAX_WORDS = len(glove.dictionary)
EMBEDDING_LENGTH = glove.no_components
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 100

model_fixed_embeds = Sequential()
model_fixed_embeds.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH, mask_zero=False, weights=[glove.word_vectors], trainable=False))
model_fixed_embeds.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
model_fixed_embeds.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model_fixed_embeds.add(crf)

model_fixed_embeds.compile(Adam(lr=0.005), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model_fixed_embeds.summary()

In [ ]:
callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=5),
         ModelCheckpoint(filepath='best_model_fixed_embeds.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model_fixed_embeds.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    batch_size=412,
    callbacks=callbacks,
    validation_split=0.1,
)

model_fixed_embeds.load_weights('best_model_fixed_embeds.h5')

Train on 2880 samples, validate on 320 samples
Epoch 1/30
2880/2880 [==============================] - 36s 12ms/step - loss: 0.8995 - crf_marginal_accuracy: 0.1284 - val_loss: 0.8082 - val_crf_marginal_accuracy: 0.8848
Epoch 2/30
2880/2880 [==============================] - 31s 11ms/step - loss: 0.7697 - crf_marginal_accuracy: 0.9253 - val_loss: 0.6789 - val_crf_marginal_accuracy: 0.9481
Epoch 3/30
2880/2880 [==============================] - 31s 11ms/step - loss: 0.6457 - crf_marginal_accuracy: 0.9438 - val_loss: 0.5597 - val_crf_marginal_accuracy: 0.9357
Epoch 4/30
2880/2880 [==============================] - 31s 11ms/step - loss: 0.5387 - crf_marginal_accuracy: 0.9198 - val_loss: 0.4755 - val_crf_marginal_accuracy: 0.9153
Epoch 5/30
2880/2880 [==============================] - 31s 11ms/step - loss: 0.4574 - crf_marginal_accuracy: 0.9300 - val_loss: 0.3930 - val_crf_marginal_accuracy: 0.9534
Epoch 6/30
2880/2880 [==============================] - 31s 11ms/step - loss: 0.3859 - crf_ma

In [ ]:
preds = model_fixed_embeds.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
print(metrics.flat_classification_report(test['encoded_ner_tags'], truncated_preds, digits=4))

              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000       150
           1     0.1954    0.0950    0.1278       179
           2     0.0448    0.6911    0.0842      1243
           3     0.0971    0.7160    0.1710       595
           4     0.2500    0.4255    0.3150       282
           5     0.1107    0.7307    0.1923       609
           6     0.2074    0.4961    0.2925       645
           7     0.2793    0.5161    0.3625       651
           8     0.0000    0.0000    0.0000        88
           9     0.0000    0.0000    0.0000         7
          10     0.0000    0.0000    0.0000       620
          11     0.5141    0.4521    0.4811       564
          12     0.1290    0.6537    0.2155       462
          13     0.3918    0.8083    0.5278       605
          15     0.9827    0.7812    0.8705    137847

    accuracy                         0.7697    144547
   macro avg     0.2135    0.4244    0.2427    144547
weighted avg     0.9454   

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
